In [ ]:
#=================================================================================================
# restAPI 테스트
# =>SBERT를 활용한 임베딩 Flask 서버 구축후, 문장을 입력 후 임베딩 값을 얻어온다.
# =>curl로 테스트 시에는 cmd창에서 아래 처럼 한글 text(안녕하세요)는 인코딩된 값으로 입력해야 함.
# curl -d "{""text"":""%EC%95%88%EB%85%95%ED%95%98%EC%84%B8%EC%9A%94""}" 
# -H "Content-Type: application/json" 
# -X POST http://127.0.0.1:9999/embed
#=================================================================================================

In [ ]:
# 임베딩 예시
# => curl -d "{""text"":""%EC%95%88%EB%85%95%ED%95%98%EC%84%B8%EC%9A%94""}" -H "Content-Type: application/json" -X POST http://127.0.0.1:9999/embed

import requests
import json

#===========================================================
# param 설정
#===========================================================

FLASK_URL = "http://127.0.0.1:9999/embed"
#===========================================================


def run_embedding_query_loop():
    while True:
        try:
            handle_query_embedding()
        except KeyboardInterrupt:
            return
        
def handle_query_embedding():
    #url
    url = FLASK_URL

    #headers
    headers = {
        "Content-Type": "application/json"
    }

    #data
    #query = "my name is bongsoo. who are you?"
    #query = "안녕하세요.지금은 curl 테스트 입니다."
    query = input("임베딩할 문장 입력: ")

    # json 구조
    temp = {
        "text": query,
        #"text2": text2,
    }

    # dict 을 json으로 변환
    data = json.dumps(temp)
    
    response = requests.post(url, headers=headers, data=data)
    print("response: ", response)
    print(type(response.text))
    print("response.text: ", response.text)
    

In [ ]:
run_embedding_query_loop()

In [ ]:
# 문장 요약후 임베딩 예시
# => curl -d "{""text"":""%EC%95%88%EB%85%95%ED%95%98%EC%84%B8%EC%9A%94""}" -H "Content-Type: application/json" -X POST http://127.0.0.1:9999/embed?min_length=10&num_sentence=2

import requests
import json

#===========================================================
# param 설정
#===========================================================
MIN_LENGTH = 10
NUM_SENTENCE = 2

FLASK_URL = f"http://127.0.0.1:9999/summarize?min_length={MIN_LENGTH}&num_sentence={NUM_SENTENCE}"
#===========================================================

def run_summarize_query_loop():
    while True:
        try:
            handle_query_summarize()
            #return
        except KeyboardInterrupt:
            return
        
def handle_query_summarize():
    #url
    url = FLASK_URL

    #headers
    headers = {
        "Content-Type": "application/json"
    }

    #data
    #query = "my name is bongsoo. who are you?"
    
    #query = '''
    #대한민국의 남서쪽에 있는 섬. 행정구역상 광역자치단체인 제주특별자치도의 관할. 
    #한국의 섬 중에서 가장 크고 인구가 많은 섬으로 면적은 1833.2㎢이다. 제주도 다음 2번째 큰 섬인 거제도의 5배 정도 된다. 인구는 약 68만 명.
    #'''
    query = input("요약할 문장 입력: ")

    # json 구조
    temp = {
        "text": query,
        #"text2": text2,
    }

    # dict 을 json으로 변환
    data = json.dumps(temp)
    
    response = requests.post(url, headers=headers, data=data)
    print("response: ", response)
    print("response.text: ", response.text)

In [ ]:
run_summarize_query_loop()

In [8]:
# ES vector 검색 예시
# => curl -d "{""text"":""%EC%95%88%EB%85%95%ED%95%98%EC%84%B8%EC%9A%94""}" -H "Content-Type: application/json" -X POST http://127.0.0.1:9999/search?eshost='http://192.168.0.27:9200/'&index='korquad-klue-2'&size=5

import requests
import json

#===========================================================
# param 설정
#===========================================================
ESURL = 'http://192.168.0.27:9200/'                 # es url
INDEX_NAME = 'korquad-albert-small-kor-sbert-v1'    # es 안덱스명(db명)
SEARCH_SIZE = 4                                     # 검색할 계수

# 검색을 위한 rest URL 설정
FLASK_URL = f"http://127.0.0.1:9999/search?esurl={ESURL}&index={INDEX_NAME}&size={SEARCH_SIZE}"
#===========================================================

def run_search_query_loop():
    while True:
        try:
            handle_query_search()
            #return
        except KeyboardInterrupt:
            return
        
def handle_query_search():
    #url
    url = FLASK_URL

    #headers
    headers = {
        "Content-Type": "application/json"
    }

    #data
    #query = "세상에게 귀여운 동물들들"
    query = input("검색할 문장 입력: ")

    # json 구조
    temp = {
        "text": query,
        #"text2": text2,
    }

    # dict 을 json으로 변환
    data = json.dumps(temp)
    
    response = requests.post(url, headers=headers, data=data)
    print("response: ", response)
    #print("response.text: ", response.text)
    
    # 들어온 응답 결과 str -> dict 로 변환하기 위해 json.loads 이용
    search_result = json.loads(response.text)
    results = search_result['result']
    print(f'count:{search_result["count"]}')
    print()
    
    # 쿼리 응답 결과에서 정리해서 보여줌.
    for result in results:
        print(f'[제목] {result["title"]}')
        print('[스코어(cross/es)] {:.4f}/{:.4f}'.format(result["cross_score"], result["es_score"]))
        print(f'[요약문]')
        print({result["summarize"]}) 
        print()
    
    
    '''
    # 쿼리 응답 결과값에서 _id, _score, _source 등을 뽑아냄
    for hit in search_result["hits"]["hits"]: 
        
        print("index:{}, type:{}".format(hit["_index"], hit["_type"]))
        print("id: {}, score: {}".format(hit["_id"], hit["_score"])) 
        
        print(f'[제목] {hit["_source"]["title"]}')
        
        print('[요약문]')
        print(hit["_source"]["summarize"]) 
        
        print()   
    '''
    

In [9]:
run_search_query_loop()

검색할 문장 입력:  하와이


response:  <Response [200]>
count:4

[제목] 과달카날_전역
[스코어(cross/es)] 1.5496/1.6129
[요약문]
{'1941년 12월 7일에 일본군은 하와이에 주둔한 미해군 태평양 함대에 대하여 진주만 공격을 감행하였다. 상륙선들은 2개의 집단으로 나뉘어서 한 집단은 과달카날에 상륙했고, 다른 집단은 투라기 플로리다와 근처의 섬들에 상륙했다.'}

[제목] 나카모리_아키나
[스코어(cross/es)] 1.5358/1.6096
[요약문]
{'나카모리 아키나(일본어: 中森 明菜, 1965년 7월 13일 ~ )는 일본의 여성 가수이자 배우이다. 우노 코레마사는 나카모리 아키나는 첫 싱글인 〈슬로우 모션〉에서 침체했다가, 두 번째 싱글 〈소녀A〉에서 히트하여 세 번째 싱글인 〈세컨드 러브〉에서 순식간에 싱글 차트 첫 1위를 달성한 것은 상당히 이례적인 일로 이 곡 이후로 꾸준히 전성기를 다져갔으며 각 싱글곡들의 이미지 변화와 진화의 속도감에 당시 일본 열도도 상당히 놀랐다라고 하였다.'}

[제목] 물고기
[스코어(cross/es)] 1.4967/1.6567
[요약문]
{'참치, 황새치, 또 상어의 몇몇 종들은 어느 정도의 온혈 적응을 나타낸다. 보다 포유류의 면역체계와 비슷하게 경골어류의 적혈구(영어: erythrocyte), 호중성 백혈구(영어: neutrophil) 그리고 과립성 백혈구(영어: granulocyte)들은, 흉선안에서 발견되는 임파구가 주된 세포형태인, 비장 안의 어디에든 존재한다고 믿어진다.'}

[제목] 하시마섬
[스코어(cross/es)] 1.4826/1.7218
[요약문]
{'하시마 섬(일본어: 端島)은 일본 나가사키현 나가사키시에 있는 무인도이다. 이 중 일부는 부적합한 채굴 조건 탓에 병에 탄광 사고나 영양실조나 병에 걸려 죽었고 도망하려고 하다가 바다에 빠져 비참하게 죽기도 했다.'}

